In [1]:
import numpy as np
import pandas as pd
import os, sys
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')


%cd C:\emobility

C:\emobility


In [2]:
replacement_bybike = pd.read_pickle(f"./dbs/output/replacement_bybike.pkl")
emission_bylink = pd.read_pickle(f"./dbs/output/emission_bylink.pkl")


In [3]:

replacement_bybike['changed_act_id'] = replacement_bybike.groupby('person').cumcount()


In [4]:
emission_bylink = pd.merge(emission_bylink, replacement_bybike[["person",'changed_act_id','tour_replacement']], left_on= ["person",'act_id'], right_on= ["person",'changed_act_id'], how="left")

In [5]:
#emission_bylink[pd.isnull(emission_bylink['tour_replacement'])]
emission_bylink_bike = emission_bylink[emission_bylink.tour_replacement==True]
print('the reduction capacity: ', emission_bylink_bike['link_emission'].sum())

the reduction capacity:  569316.1767673519


In [6]:
emission_bylink_car = emission_bylink[emission_bylink.tour_replacement==False]
print('the reduced emission: ', emission_bylink_car['link_emission'].sum())

the reduced emission:  1931319.0316369685


In [7]:
emission_bylink['link_emission'].sum()

2500635.208404324

In [8]:
emission_bylink.tour_replacement.value_counts()

False    42063216
True     27736811
Name: tour_replacement, dtype: int64

In [9]:
print('replaced trips total distance:', emission_bylink['link_distance'][emission_bylink.tour_replacement==True].sum())
print('not replaced trips total distance:',emission_bylink['link_distance'][emission_bylink.tour_replacement==False].sum())

print('replaced trips total distance ratio:', emission_bylink['link_distance'][emission_bylink.tour_replacement==True].sum()/emission_bylink['link_distance'].sum())


replaced trips total distance: 3655018.4338806206
not replaced trips total distance: 12660595.957870563
replaced trips total distance ratio: 0.2240196627672513


In [10]:
emission_bylink['num_Deso']= emission_bylink['Deso'].str.len()

In [11]:
emission_bylink['link_emission']= emission_bylink['link_emission'].div(emission_bylink['num_Deso'])

In [12]:
emission_bylink= emission_bylink.loc[emission_bylink.index.repeat(emission_bylink.num_Deso)]

In [13]:
emission_bylink['num_Deso']= emission_bylink.groupby(['person','seq']).cumcount()


In [14]:
def link_todeso(row):
    if len(row['Deso']) == 1:
        Deso = row['Deso'][0]
    else:
        Deso = row['Deso'][row['num_Deso']]
    return Deso

In [15]:
emission_bylink['Deso'] = emission_bylink.apply( lambda row: link_todeso(row), axis=1)

In [16]:
emission_bydeso = emission_bylink.groupby(['Deso'])['link_emission'].sum().reset_index()

### Visualization

In [17]:
import geopandas as gpd



DeSO = gpd.read_file(f'./dbs/emission/deso_statistik_shp/Fordon_region.shp')
DeSO = DeSO[['Deso','geometry']]

In [18]:
gdf=DeSO.merge(emission_bydeso,on="Deso")
gdf= gdf[gdf['Deso'].str[:2]=='14']


In [19]:
print('the total emission within VG: ', gdf.link_emission.sum())


the total emission within VG:  2175412.652603938


In [21]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable


fig, ax = plt.subplots(1, 1, figsize=(12, 12), dpi=250)

divider = make_axes_locatable(ax)
cax = divider.append_axes("bottom", size="5%", pad=0.1)

gdf.plot(column='link_emission',
         ax=ax,
         cmap='viridis',
         legend=True,
         cax=cax,
         legend_kwds={'label': "Greenhouse gas emissions from passenger cars by the driven DeSO zones (in kg CO2 eq.)", 'orientation': "horizontal"})

#cax.tick_params(labelsize='12')

#plt.show()
plt.savefig(r'.\figures\p-emission_abs.jpg')

In [22]:
emission_bylink_reduction = emission_bylink[emission_bylink.tour_replacement==True]
emission_bylink_reduction = emission_bylink_reduction.groupby(['Deso'])['link_emission'].sum().reset_index()



In [23]:
gdf_reduction = DeSO.merge(emission_bylink_reduction, on="Deso")
gdf_reduction = gdf_reduction[gdf_reduction['Deso'].str[:2] == '14']


print('the reduction in emission within VG: ', gdf_reduction.link_emission.sum())


the reduction in emission within VG:  556184.589955094


In [24]:



fig, ax = plt.subplots(1, 1, figsize=(12, 12), dpi=250)

divider = make_axes_locatable(ax)
cax = divider.append_axes("bottom", size="5%", pad=0.1)

gdf_reduction.plot(column='link_emission',
         ax=ax,
         cmap='viridis',
         legend=True,
         cax=cax,
         legend_kwds={'label': "Reduction in greenhouse gas emissions from passenger cars by the driven DeSO zones (in kg CO2 eq.)", 'orientation': "horizontal"})

#plt.show()
plt.savefig(r'.\figures\p-emission_reduction_abs.jpg')


In [25]:
gdf_percentage_reduction = gdf.merge(gdf_reduction[['Deso','link_emission']], on="Deso")
gdf_percentage_reduction['percentage'] = (gdf_percentage_reduction['link_emission_y']/gdf_percentage_reduction['link_emission_x'])*100


fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=250)

divider = make_axes_locatable(ax)
cax = divider.append_axes("bottom", size="5%", pad=0.1)

gdf_percentage_reduction.plot(column='percentage',
         ax=ax,
         cmap='viridis',
         legend=True,
         cax=cax,
         legend_kwds={'label': "Reduction in greenhouse gas emissions from passenger cars by the driven DeSO zones (in %)", 'orientation': "horizontal"})

#plt.show()
plt.savefig(r'.\figures\p-emission_reduction_per.jpg')



In [26]:
emission_byperson = emission_bylink.groupby(['person'])['link_emission'].sum().reset_index()


emission_byperson_reduction = emission_bylink[emission_bylink.tour_replacement==True]
emission_byperson_reduction = emission_byperson_reduction.groupby(['person'])['link_emission'].sum().reset_index()


In [27]:
df_act = pd.read_csv(f'./dbs/agents/1_activity_plans.csv')
df_act = df_act[df_act.act_purpose=='home']
df_act.drop_duplicates(subset=['person'], inplace=True)


In [28]:
emission_byperson = pd.merge(emission_byperson, df_act[['person','deso']], on='person', how="left")

emission_byperson_reduction = pd.merge(emission_byperson_reduction, df_act[['person','deso']], on='person', how="left")

In [29]:
emission_byresidentialzone = emission_byperson.groupby(['deso'])['link_emission'].sum().reset_index()

emission_byresidentialzone_reduction = emission_byperson_reduction.groupby(['deso'])['link_emission'].sum().reset_index()

In [30]:
gdf_residentialzone = gdf[['Deso','geometry']].merge(emission_byresidentialzone[['deso','link_emission']], left_on="Deso", right_on="deso")
gdf_residentialzone = gdf_residentialzone.merge(emission_byresidentialzone_reduction[['deso','link_emission']], left_on="Deso", right_on="deso")


In [31]:

gdf_residentialzone['percentage'] = (gdf_residentialzone['link_emission_y']/gdf_residentialzone['link_emission_x'])*100


fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=250)

divider = make_axes_locatable(ax)
cax = divider.append_axes("bottom", size="5%", pad=0.1)
ax.set_axis_off()
x, y, arrow_length = 0.9, 0.2, 0.1
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=20,
            xycoords=ax.transAxes)



gdf_residentialzone.plot(column='percentage',
         ax=ax,
         cmap='viridis',
         legend=True,
         cax=cax,
         legend_kwds={'label': "Reduction in greenhouse gas emissions from passenger cars by the agents' residential DeSO zones (in %)", 'orientation': "horizontal"})

#plt.show()
plt.savefig(r'.\figures\p-emission_reduction_per_residential.jpg')



# bar chart by population density

In [32]:
DeSO_age = gpd.read_file(f'./dbs/emission/deso_statistik_shp/Bef_Alder_region.shp')

Muni_names = pd.read_csv(f'./dbs/emission/deso_statistik_shp/muni-name.csv', delimiter=";", dtype= {"KNKOD":str, 'LANDAREAKM':float})




DeSO_age= DeSO_age[DeSO_age['Deso'].str[:2]=='14']

DeSO_age['municipality'] = DeSO_age['Deso'].str[:4]

DeSO_age = DeSO_age.groupby(['municipality'])['Totalt', 'area_sw'].sum().reset_index()

DeSO_age[ 'area_sw'] = DeSO_age[ 'area_sw']/ 10**6


DeSO_age = pd.merge(DeSO_age, Muni_names, left_on='municipality' , right_on='KNKOD')


DeSO_age['LANDAREAKM'] = np.where(DeSO_age['LANDAREAKM'] == 0.00000 , DeSO_age['area_sw'], DeSO_age['LANDAREAKM'])

DeSO_age['pop_density'] =DeSO_age['Totalt']/ DeSO_age['LANDAREAKM']


In [33]:
gdf_residentialzone['municipality'] = gdf_residentialzone['Deso'].str[:4]

gdf_residentialmuni = gdf_residentialzone.groupby(['municipality'])['link_emission_x','link_emission_y'].sum().reset_index()

gdf_residentialmuni['percentage'] = (gdf_residentialmuni['link_emission_y']/gdf_residentialmuni['link_emission_x'])*100

gdf_residentialmuni['remaining_percentage'] = 100 -gdf_residentialmuni['percentage']


DeSO_age = pd.merge(DeSO_age, gdf_residentialmuni, left_on='municipality' , right_on='municipality')



In [34]:
#https://matplotlib.org/stable/gallery/subplots_axes_and_figures/two_scales.html

import matplotlib.pyplot as plt

# Data
groups = DeSO_age['short_name'].tolist()
values2 =DeSO_age['percentage']
values1 =DeSO_age['remaining_percentage']
values3 =DeSO_age['pop_density']

#
fig, ax = plt.subplots(figsize=(16,8))
#
# # Stacked bar chart
ax.bar(groups, values1, label = "Remaining emissions")
ax.bar(groups, values2, bottom = values1, label = "Reduction in emissions")

ax1 = ax.twinx()  # instantiate a second axes that shares the same x-axis

ax1.plot(groups, values3, color='r', marker='o', label='Population / Area')
ax1.set_ylabel('Number of ppl / Area (km2)', size = 12, color='r')
ax1.tick_params(axis='y', labelcolor='r')
ax1.legend(loc="lower left",fontsize=12)




ax.legend(loc="upper right",fontsize=12)
ax.set_ylabel('Share (%)', size = 12)

ax.set_xlabel('Municipalities', size = 15)
ax.tick_params(axis='x', labelrotation=90)




plt.savefig(r'.\figures\reduction_barchart_byMuni.jpg')

In [35]:

DeSO_age.sort_values(by='pop_density', ascending=False, inplace=True)


# Data
groups = DeSO_age['short_name'].tolist()
values2 =DeSO_age['percentage']



# Figure Size
fig, ax = plt.subplots(figsize =(18, 14))


ax.bar(groups, values2,  width = 0.5)


#ax.grid(zorder=0)


# Add padding between axes and labels
ax.xaxis.set_tick_params(pad = 1, labelrotation=90, labelsize=15)
ax.yaxis.set_tick_params(pad = 10, labelsize=16)
#ax.set_yticks(np.range(0, 80, step=10), size = 12)
ax.set_ylabel('Share (%)', size = 18)

plt.yticks(range(0, 70, 10))


# Show Plot
plt.savefig(r'.\figures\reduction_barchart_byMuni_ordered.jpg')


In [36]:

DeSO_age.sort_values(by='pop_density', ascending=True, inplace=True)


# Data
groups = DeSO_age['short_name'].tolist()
values2 =DeSO_age['percentage']



# Figure Size
fig, ax = plt.subplots(figsize =(15, 18))


ax.barh(groups, values2)


#ax.grid(zorder=0)


# Add padding between axes and labels
ax.yaxis.set_tick_params(pad = 1, labelrotation=0, labelsize=18)
ax.xaxis.set_tick_params(pad = 10, labelsize=18)
#ax.set_yticks(np.range(0, 80, step=10), size = 12)
ax.set_xlabel('Share (%)', size = 20)


plt.xticks(range(0, 60, 10))


# Show Plot
plt.savefig(r'.\figures\reduction_barchart_byMuni_orderedh.jpg')
